In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler


In [2]:
# loading pickle file for standardization
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [3]:
#Loading ANN model 
from keras.models import load_model
import keras.metrics

# Load your model and specify the custom objects
model = load_model('temp1.h5', custom_objects={'mse': keras.metrics.mean_squared_error})


In [4]:
scaler

MinMaxScaler()

In [6]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,404 (9.39 KB)

 Trainable params: 2,402 (9.38 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [7]:
#Reading Validation dataset
data = pd.read_csv('Validation Dataset.csv')

In [8]:
data.head()

,SI,Date,Unnamed: 2,Temperature(1ft),Temperature(2ft),Temperature(3ft),Temperature(4ft),Rainfall(mm),Air Temp(F),Air Temp C
0,1,5/1/2019,NaN,8.43,8.24,8.00,7.60,1.778,48.8,9.333
1,2,5/2/2019,NaN,8.95,8.36,8.01,7.60,4.826,53.7,12.056
2,3,5/3/2019,NaN,10.19,8.95,8.22,7.65,3.810,55.6,13.111
3,4,5/4/2019,NaN,11.24,9.67,8.62,7.78,8.128,53.3,11.833
4,5,5/5/2019,NaN,12.02,10.34,9.09,7.98,0.000,52.3,11.278


##### Feature Engineering

In [9]:
data.drop(columns=['SI','Unnamed: 2','Air Temp(F)'],axis=1,inplace=True)

In [10]:
data.head()

,Date,Temperature(1ft),Temperature(2ft),Temperature(3ft),Temperature(4ft),Rainfall(mm),Air Temp C
0,5/1/2019,8.43,8.24,8.00,7.60,1.778,9.333
1,5/2/2019,8.95,8.36,8.01,7.60,4.826,12.056
2,5/3/2019,10.19,8.95,8.22,7.65,3.810,13.111
3,5/4/2019,11.24,9.67,8.62,7.78,8.128,11.833
4,5/5/2019,12.02,10.34,9.09,7.98,0.000,11.278


In [11]:
# Convert 'Date' to datetime 
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Create the 'day_of_year' column
data['day_of_year'] = data['Date'].dt.dayofyear


In [12]:
data.head()

,Date,Temperature(1ft),Temperature(2ft),Temperature(3ft),Temperature(4ft),Rainfall(mm),Air Temp C,day_of_year
0,2019-05-01,8.43,8.24,8.00,7.60,1.778,9.333,121
1,2019-05-02,8.95,8.36,8.01,7.60,4.826,12.056,122
2,2019-05-03,10.19,8.95,8.22,7.65,3.810,13.111,123
3,2019-05-04,11.24,9.67,8.62,7.78,8.128,11.833,124
4,2019-05-05,12.02,10.34,9.09,7.98,0.000,11.278,125


In [13]:
#converting mm to inch

data['Rainfall(inch)'] = data['Rainfall(mm)']/25.4

In [ ]:
#Taking relevant feature for prediction

In [14]:
data = data[['Air Temp C','Rainfall(inch)','day_of_year']]

data.head()

,Air Temp C,Rainfall(inch),day_of_year
0,9.333,0.07,121
1,12.056,0.19,122
2,13.111,0.15,123
3,11.833,0.32,124
4,11.278,0.00,125


In [15]:
#Standardization

validation_data_scaled = scaler.transform(data)

In [16]:
#prediction

predicted_values = model.predict(validation_data_scaled)

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [17]:
predicted_values

array([[18.700241, 21.672953],
       [19.207489, 21.692139],
       [19.398432, 21.701344],
       ...,
       [17.798841, 21.627127],
       [18.015509, 21.646803],
       [18.361906, 21.638937]], dtype=float32)

In [18]:
#creating dataframe for predicted values
df = pd.DataFrame(predicted_values, columns=['Temp_Pred(0.9m)', 'Temp_Pred(1.8m)'])

In [19]:
df.head()

,Temp_Pred(0.9m),Temp_Pred(1.8m)
0,18.700241,21.672953
1,19.207489,21.692139
2,19.398432,21.701344
3,19.406530,21.872972
4,19.221674,21.823034


In [20]:
#concate both dataframes

data = pd.concat([data, df], axis=1)

In [22]:
data.head()

,Air Temp C,Rainfall(inch),day_of_year,Temp_Pred(0.9m),Temp_Pred(1.8m)
0,9.333,0.07,121,18.700241,21.672953
1,12.056,0.19,122,19.207489,21.692139
2,13.111,0.15,123,19.398432,21.701344
3,11.833,0.32,124,19.406530,21.872972
4,11.278,0.00,125,19.221674,21.823034


In [23]:
data.tail()

,Air Temp C,Rainfall(inch),day_of_year,Temp_Pred(0.9m),Temp_Pred(1.8m)
1091,11.389,0.00,116,18.538698,21.270189
1092,4.000,0.05,117,17.723274,21.617914
1093,4.500,0.01,118,17.798841,21.627127
1094,5.611,0.00,119,18.015509,21.646803
1095,7.667,0.00,120,18.361906,21.638937


In [24]:
data

,Air Temp C,Rainfall(inch),day_of_year,Temp_Pred(0.9m),Temp_Pred(1.8m)
0,9.333,0.07,121,18.700241,21.672953
1,12.056,0.19,122,19.207489,21.692139
2,13.111,0.15,123,19.398432,21.701344
3,11.833,0.32,124,19.406530,21.872972
4,11.278,0.00,125,19.221674,21.823034
...,...,...,...,...,...
1091,11.389,0.00,116,18.538698,21.270189
1092,4.000,0.05,117,17.723274,21.617914
1093,4.500,0.01,118,17.798841,21.627127
1094,5.611,0.00,119,18.015509,21.646803
